In [ ]:
"""
TODO:
* Figure out 1-row tables (pg 847)
* Figure out wonky tables with extra chars (pg 577)
"""

### Prepping the PDF Parser

In [ ]:
#install the pdf extract library

!pip install pdfminer3

In [126]:
#import the necessary packages for pulling data out of PDFs
import pandas as pd
import io

from pdfminer3.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer3.converter import TextConverter, XMLConverter
from pdfminer3.layout import LAParams
from pdfminer3.pdfpage import PDFPage

In [127]:
#build a function that pulls out the data from the pdf into a giant string

def convert_pdf_to_txt(path,pg):
    if isinstance(pg, int):
        pg=[pg-1]
    else:
        print('please pass an integer for your pagenumber')
        return
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = pg

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,
                                  password=password,
                                  caching=caching,
                                  check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

### >>>Put in your Variables here<<<

In [415]:
fl = 'NOVA_L998'
page = 571

#----------#
ext = '.PDF'
file = fl+ext

### This gets the pdf page and splits everything into lists

In [405]:
### point the file at the function and parse the specified page
p = convert_pdf_to_txt(file, page)
#print(p)

pp = p.split('\n\n')
pp2 = []
lp = 0
for l in range(len(pp)):   
    pp2.append(pp[l].split('\n'))

#pp2

### This organizes the data puts it into a dataframe

In [408]:
### Cleanup the dataset into column headers, metadata, and real data
col = []
data = []
meta = []
for l in range(len(pp2)):
    if pp2[l][0] == 'No.':
        col.append('No Ref')
    elif len(pp2[l]) == 2 and pp2[l][0] == 'Parts':
        col.append('Parts Num')
    elif len(pp2[l]) == 1 and pp2[l][0] == 'Description':
        col.append('Description')
    elif len(pp2[l]) == 1 and pp2[l][0] == 'Quantity':
        col.append('Quantity')
    elif len(pp2[l]) == 1 and pp2[l][0] != '\x0c':
        meta.append(pp2[l])
    elif pp2[l][0] != '\x0c':
        data.append(pp2[l])
    #print(len(pp2[l]), pp2[l][0] == 'No.')

### create a df for the data, add column names & add a tmp column for joining metadata
df = pd.DataFrame(data).transpose()
df.columns = col
df['tmp'] = 1
#df

### create a df for the metadata & add tmp col for joining to base data
dfm = pd.DataFrame(meta).transpose()
dfm.columns = ['Category','Sub-Category','Localisation','REV' ]
dfm['tmp'] = 1
#dfm

### join the metadata to the base data and re-order columns
data = df.merge(dfm, on='tmp')
data.drop(['tmp'], axis=1, inplace=True)
data = data[['No Ref', 'Parts Num', 'Description', 'Quantity','Category', 'Sub-Category','Localisation','REV']]
data

,No Ref,Parts Num,Description,Quantity,Category,Sub-Category,Localisation,REV
0,01,REF,REFERENCE TO PAGE N87400-02,01,ENGINE AND COOLING,BELT GUARD INST,09-403.37X4,18AL17
1,02,N87863-02,SUPPORT,01,ENGINE AND COOLING,BELT GUARD INST,09-403.37X4,18AL17
2,03,2303453,NEOPRENE WASHER 1-1/4 OD X 13/32 ID,01,ENGINE AND COOLING,BELT GUARD INST,09-403.37X4,18AL17
3,04,N45837,RIVET NUT 0.5 / 3.0,04,ENGINE AND COOLING,BELT GUARD INST,09-403.37X4,18AL17
4,05,N17905,LOCK WASHER M8,07,ENGINE AND COOLING,BELT GUARD INST,09-403.37X4,18AL17
5,06,N21101,WASHER OVERSIZE M8,07,ENGINE AND COOLING,BELT GUARD INST,09-403.37X4,18AL17
6,07,N11168,"BOLT M8 X 1,25 X 25 GR8",07,ENGINE AND COOLING,BELT GUARD INST,09-403.37X4,18AL17
7,08,N44888,LOCKNUT NYLON M8,02,ENGINE AND COOLING,BELT GUARD INST,09-403.37X4,18AL17
8,REF,R085023,SEALANT NEVER SEIZE,QR,ENGINE AND COOLING,BELT GUARD INST,09-403.37X4,18AL17


### Write the data to a CSV File

In [417]:
data.to_csv('page '+str(page)+' from '+fl+'.csv', index=False)